In [1]:
import os

In [2]:
%pwd

'c:\\Users\\RICH-FILES\\Desktop\\ml\\AI-powered-Bank-Product-Recommender-Chatbot\\research'

In [3]:
os.chdir("../.")

In [4]:
%pwd

'c:\\Users\\RICH-FILES\\Desktop\\ml\\AI-powered-Bank-Product-Recommender-Chatbot'

In [5]:
project_dir = "C:/Users/RICH-FILES/Desktop/ml/AI-powered-Bank-Product-Recommender-Chatbot"
os.chdir(project_dir)

In [6]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    raw_data_dir: Path
    customers_csv: Path
    products_csv: Path
    data_file: Path
    

In [7]:
from BankProducts.constants import *
from BankProducts.utils.common import read_yaml, create_directories

In [8]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath: str = CONFIG_FILE_PATH,    
        params_filepath: str = PARAMS_FILE_PATH,
        #schema_filepath: str = SCHEMA_FILE_PATH,
        ):
       
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        #self.schema = read_yaml(schema_filepath)        
    
        create_directories([self.config.artifacts_root])
    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
    
        create_directories([self.config.artifacts_root])
        
        data_ingestion_config = DataIngestionConfig(
            root_dir=Path(config.root_dir),
            raw_data_dir=Path(config.raw_data_dir),
            customers_csv=Path(config.customers_csv),
            products_csv=Path(config.products_csv),
             data_file=Path(config.data_file)
        
        )
        
        return data_ingestion_config

In [9]:
import os
import logging
import sqlite3
import pandas as pd
from pathlib import Path
from sqlalchemy import create_engine
from BankProducts import logger

In [ ]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def extract_and_save_data(self):
        """Extract customers and products data from SQLite DB and save as CSV."""
        logging.info(f"Connecting to database: {self.config.data_file}")
        conn = sqlite3.connect(self.config.data_file)

        try:
            customers_df = pd.read_sql_query(f"SELECT * FROM {self.config.customers_csv}", conn)
            products_df = pd.read_sql_query(f"SELECT * FROM {self.config.products_csv}", conn)
            logging.info(f"Successfully read tables {self.config.customers_csv} and {self.config.products_csv}")
        finally:
            conn.close()
            logging.info("Database connection closed.")

        # Ensure directories exist
        Path(self.config.raw_data_dir).mkdir(parents=True, exist_ok=True)

        # Save to CSV
        customers_df.to_csv(self.config.customers_csv, index=False)
        products_df.to_csv(self.config.products_csv, index=False)
        logging.info(f"Customers data saved to: {self.config.customers_csv}")
        logging.info(f"Products data saved to: {self.config.products_csv}")
        
        return customers_df, products_df

In [11]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.extract_and_save_data()
except Exception as e:
    logger.exception(e)
    raise e


[2025-05-18 14:35:56,013: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-05-18 14:35:56,017: INFO: common: yaml file: params.yaml loaded successfully]
[2025-05-18 14:35:56,020: INFO: common: created directory at: artifacts]
[2025-05-18 14:35:56,022: INFO: common: created directory at: artifacts]
[2025-05-18 14:35:56,023: INFO: 4111995723: Connecting to database: artifacts\data_generation\bank_products_recommender.db]
[2025-05-18 14:35:56,026: INFO: 4111995723: Database connection closed.]
[2025-05-18 14:35:56,028: ERROR: 930908772: 'DataIngestionConfig' object has no attribute 'customers_table']
Traceback (most recent call last):
  File "C:\Users\RICH-FILES\AppData\Local\Temp\ipykernel_13980\930908772.py", line 5, in <module>
    data_ingestion.extract_and_save_data()
  File "C:\Users\RICH-FILES\AppData\Local\Temp\ipykernel_13980\4111995723.py", line 11, in extract_and_save_data
    customers_df = pd.read_sql_query(f"SELECT * FROM {self.config.customers_table}",

AttributeError: 'DataIngestionConfig' object has no attribute 'customers_table'